yatb - yet another test bench

Todo: Create a common test tool for all models. It should use a translator as input, and output bleu score for the test-set of choice.

In [1]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from utils.preprocess import fields_for_rule_text
from torchtext.legacy.data import Field

c:\Users\ddw\Anaconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
SRC, TRG = fields_for_rule_text()
fields = {'src-rule': ('src', SRC), 'trg-rule': ('trg', TRG)}
fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
data = TestSets.load(fields)

In [11]:
print(vars(data.examples[0]))

{'src': ['Whenever', 'another', 'artifact', 'or', 'creature', 'you', 'control', 'is', 'put', 'into', 'a', 'graveyard', 'from', 'the', 'battlefield,', 'put', 'an', 'oil', 'counter', 'on', 'Vat', 'of', 'Rebirth.\n{2}{B},', '{T},', 'Remove', 'four', 'oil', 'counters', 'from', 'Vat', 'of', 'Rebirth:', 'Return', 'target', 'creature', 'card', 'from', 'your', 'graveyard', 'to', 'the', 'battlefield.', 'Activate', 'only', 'as', 'a', 'sorcery.'], 'trg': ['每当另一个由你操控的神器或生物从战场进入坟墓场时，在再生缸上放置一个烁油指示物。', '{2}{B}，{T}，从再生缸上移去四个烁油指示物：将目标生物牌从你的坟墓场移回战场。只能于法术时机起动。']}


In [21]:
from models.card_name_detector.definition import TrainedDetector

D = TrainedDetector()
D.annotate('when xbp, NKU\'s hope enters the battlefield, search your library for a card named ddw, xbp\'s minion, put it into your hand.',
           threshold=1e-3)

path: d:\ddw\school\大三下\语音信息处理技术\期末作业\code\mtg-cards-translation\models\card_name_detector
['when', 'xbp', ',', 'nku', "'s", 'hope', 'enters', 'the', 'battlefield', ',', 'search', 'your', 'library', 'for', 'a', 'card', 'named', 'ddw', ',', 'xbp', "'s", 'minion', ',', 'put', 'it', 'into', 'your', 'hand', '.']


" when <xbp , nku 's hope> enters the battlefield , search your library for a card named <ddw , xbp 's minion> , put it into your hand ."

In [19]:
def sentencize(text: str):
    ignore = {' ', '(', ')', '\n'}
    while len(text) and text[0] in ignore:
        text = text[1:]
    if len(text) == 0:
        return []
    
    r = 0
    delims = {'.', '\n', '('}
    while r < len(text) and text[r] not in delims:
        r += 1
    
    if r < len(text) and text[r] == '.':
        return [text[:r + 1]] + sentencize(text[r + 1:])
    return [text[:r]] + sentencize(text[r:])

In [20]:
class CardTranslator:
    def __init__(self, sentencize, sent_translator, preprocess=None, postprocess=None) -> None:
        self.sentencize = sentencize
        self.sent_translator = sent_translator
        self.preprocess = preprocess
        self.postprocess = postprocess
    
    def translate(self, text: str)->str:
        sents = self.sentencize(text)
        result = []
        for sent in sents:
            if self.preprocess:
                sent = self.preprocess(sent)
            sent = self.sent_translator.translate(sent)
            if self.postprocess:
                sent = self.postprocess(sent)
            result.append(sent)
        return ' '.join(result)

['Toxic 2 ', 'Players dealt combat damage by this creature also get two poison counters.']
['Whenever you cast a noncreature spell, put an oil counter on Atmosphere Surgeon.', 'Remove an oil counter from Atmosphere Surgeon: Target creature gains flying until end of turn.', 'Activate only as a sorcery.']
['Enchant creature', 'When Mesmerizing Dose enters the battlefield, tap enchanted creature, then proliferate.', 'Choose any number of permanents and/or players, then give each another counter of each kind already there.', "Enchanted creature doesn't untap during its controller's untap step."]
['For Mirrodin! ', 'When this Equipment enters the battlefield, create a 2/2 red Rebel creature token, then attach this to it.', 'Whenever Blade of Shared Souls becomes attached to a creature, for as long as Blade of Shared Souls remains attached to it, you may have that creature become a copy of another target creature you control.', 'Equip {2}']
['Compleated ', '{G/P} can be paid with {G} or 2 li